In [1]:
from sklearn.linear_model import Lasso, LassoCV, RidgeCV
import numpy as np

class HAL:
    def __init__(self, *args, **kwargs):
        self.lasso = LassoCV(*args, **kwargs)

    def _basis_products(self, arr, index=0, current=None, result=None):
        if result is None:
            result = []
        if current is None:
            current = np.ones_like(arr[0], dtype=bool)

        if index == len(arr):
            result.append(current)
        else:
            self._basis_products(arr, index + 1, current & arr[index], result)
            self._basis_products(arr, index + 1, current, result)

        return result

    def _bases(self, X):
        one_way_bases = np.stack([
            np.less.outer(self.knots[:,j], X[:,j])
            for j in range(self.knots.shape[1])
        ])
        bases = self._basis_products(one_way_bases)
        return np.concatenate(bases[:-1]).T

    def fit(self, X, Y):
        self.knots = X
        self.lasso.fit(self._bases(X), Y) # (HᵀH + λI)^{-1}HᵀY = β

    def predict(self, X):
        return self.lasso.predict(self._bases(X)) # Hβ


In [2]:
import numpy as np

class dgp:
    @classmethod
    def f(cls, X):
        return -0.5*X[:,0] + (X[:,1] * X[:,0]**2) / 2.75 + X[:,1]

    @classmethod
    def gen(cls, n):
        X = np.column_stack((
            np.random.uniform(-4, 4, n),
            np.random.binomial(1, 0.5, n),
        ))
        Y = cls.f(X) + np.random.normal(0, 1, n)
        return X, Y

In [3]:
X,Y = dgp.gen(100)
X_test, Y_test = dgp.gen(10000)

In [4]:
model = HAL()
model.fit(X,Y)
model.lasso.alphas_

array([0.75953377, 0.70834373, 0.66060374, 0.61608126, 0.57455944,
       0.53583606, 0.4997225 , 0.46604288, 0.43463315, 0.40534033,
       0.37802175, 0.35254435, 0.32878404, 0.30662509, 0.28595959,
       0.26668687, 0.24871307, 0.23195064, 0.21631794, 0.20173884,
       0.18814231, 0.17546215, 0.16363659, 0.15260803, 0.14232276,
       0.13273068, 0.12378508, 0.11544238, 0.10766195, 0.10040589,
       0.09363887, 0.08732793, 0.08144232, 0.07595338, 0.07083437,
       0.06606037, 0.06160813, 0.05745594, 0.05358361, 0.04997225,
       0.04660429, 0.04346331, 0.04053403, 0.03780217, 0.03525443,
       0.0328784 , 0.03066251, 0.02859596, 0.02666869, 0.02487131,
       0.02319506, 0.02163179, 0.02017388, 0.01881423, 0.01754622,
       0.01636366, 0.0152608 , 0.01423228, 0.01327307, 0.01237851,
       0.01154424, 0.01076619, 0.01004059, 0.00936389, 0.00873279,
       0.00814423, 0.00759534, 0.00708344, 0.00660604, 0.00616081,
       0.00574559, 0.00535836, 0.00499723, 0.00466043, 0.00434

In [5]:
model.lasso.alpha_

0.02319506383965411

In [ ]:
model = HAL()
model.fit(X,Y)

In [ ]:
Ŷ = model.predict(X_test)

In [ ]:
np.mean((Y_test - Ŷ)**2)

1.354553701520526